# Deep Learning Classification of Anomaly Peaks with 1D resnet

In [ ]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from sklearn import preprocessing
from resnet import ResNet1D
from tqdm import tqdm

sys.path.insert(1, "../")

from datasets import fdomDataset

In [ ]:
# Hyperparams
WINDOW_SIZE = 15 # the size of each data segment

In [ ]:
# Paths to data files
fdom_raw_data = (
    "../Data/converted_data/julian_format/fDOM_raw_10.1.2011-9.4.2020.csv"
)
stage_raw_data = "../Data/converted_data/julian_format/stage_10.1.11-1.1.19.csv"
turb_raw_data = (
    "../Data/converted_data/julian_format/turbidity_raw_10.1.2011_9.4.2020.csv"
)

fdom_labeled = "../Data/labeled_data/ground_truths/fDOM/fDOM_all_julian_0k-300k.csv"

In [ ]:
# get device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Create dataset and dataloaders

In [ ]:
# setup k-fold validation here
def train_val_dataset(dataset, val_split=0.10):
    pass

In [ ]:
# create dataset

classes = ["NAP", "FSK", "FPT", "PLP", "PP", "SKP"]
le = preprocessing.LabelEncoder()

targets = le.fit_transform(classes)

dataset = fdomDataset(
    le,
    fdom_raw_data,
    stage_raw_data,
    turb_raw_data,
    fdom_labeled,
    window_size=WINDOW_SIZE
)

# TODO: split data into training and testing

In [ ]:
# create dataloader
trainloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)


In [ ]:
# create model
model = ResNet1D(
    in_channels=WINDOW_SIZE * 2 + 1,
    base_filters=64,
    kernel_size=16,
    stride=2,
    n_block=48,
    groups=1, # check this
    n_classes=len(classes),
    downsample_gap=6,
    increasefilter_gap=12,
    verbose=False
)

# model.to(device)

model = model.float()

In [ ]:
# Optimizer/criterion
optimizer = optim.Adam(model.parameters(), lr=1e-3)

criterion = nn.CrossEntropyLoss()
all_loss = []


## Train Model

In [ ]:
prog_bar = tqdm(trainloader, desc='Training', leave=False)
for i, batch in enumerate(prog_bar):
    x = batch[0]

    # squeeze y to flatten predictions into 1d tensor
    y = batch[1].squeeze()

    pred = model(x.float())

    loss = criterion(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    all_loss.append(loss.item())


In [ ]:
# Test model
